In [2]:
!unzip ./dataset_dr_preprocessed.zip

Archive:  ./dataset_dr_preprocessed.zip
   creating: dataset_dr_preprocessed/
  inflating: dataset_dr_preprocessed/.DS_Store  
  inflating: __MACOSX/dataset_dr_preprocessed/._.DS_Store  
   creating: dataset_dr_preprocessed/c0/
   creating: dataset_dr_preprocessed/c1/
   creating: dataset_dr_preprocessed/c3/
   creating: dataset_dr_preprocessed/c4/
   creating: dataset_dr_preprocessed/c2/
  inflating: dataset_dr_preprocessed/c0/4a5a6efc0bef.png  
  inflating: dataset_dr_preprocessed/c0/17f6c7072f61.png  
  inflating: dataset_dr_preprocessed/c0/94076a9fb9b5.png  
  inflating: dataset_dr_preprocessed/c0/b09101adb478.png  
  inflating: dataset_dr_preprocessed/c0/d74ccc796517.png  
  inflating: dataset_dr_preprocessed/c0/a821b6ecef33.png  
  inflating: dataset_dr_preprocessed/c0/345b1f0abbba.png  
  inflating: dataset_dr_preprocessed/c0/ae5d31979f19.png  
  inflating: dataset_dr_preprocessed/c0/283c3aeba594.png  
  inflating: dataset_dr_preprocessed/c0/90a786abe58e.png  
  inflating: datas

In [1]:
!pip install grad-cam

     |████████████████████████████████| 4.5 MB 23.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for grad-cam: filename=grad_cam-1.3.7-py3-none-any.whl size=25953 sha256=4ad53ac5b596ac7604a1f6a55f7af51d95682d882f3f8c19d16ed5bd25b59b25
  Stored in directory: /root/.cache/pip/wheels/30/ab/9c/53c523785edffdc6c61755cf82e0dac3342d0d36190c187894
Successfully built grad-cam


In [3]:
import torch
import torch.nn as  nn
import torch.nn.functional as F


class Bottleneck(nn.Module):
    expansion = 4
    def __init__(self, in_channels, out_channels, i_downsample=None, stride=1):
        super(Bottleneck, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0)
        self.batch_norm1 = nn.BatchNorm2d(out_channels)
        
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.batch_norm2 = nn.BatchNorm2d(out_channels)
        
        self.conv3 = nn.Conv2d(out_channels, out_channels*self.expansion, kernel_size=1, stride=1, padding=0)
        self.batch_norm3 = nn.BatchNorm2d(out_channels*self.expansion)
        
        self.i_downsample = i_downsample
        self.stride = stride
        self.relu = nn.ReLU()
        
    def forward(self, x):
        identity = x.clone()
        x = self.relu(self.batch_norm1(self.conv1(x)))
        
        x = self.relu(self.batch_norm2(self.conv2(x)))
        
        x = self.conv3(x)
        x = self.batch_norm3(x)
        
        #downsample if needed
        if self.i_downsample is not None:
            identity = self.i_downsample(identity)
        #add identity
        x+=identity
        x=self.relu(x)
        
        return x

class Block(nn.Module):
    expansion = 1
    def __init__(self, in_channels, out_channels, i_downsample=None, stride=1):
        super(Block, self).__init__()
       

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, stride=stride, bias=False)
        self.batch_norm1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, stride=stride, bias=False)
        self.batch_norm2 = nn.BatchNorm2d(out_channels)

        self.i_downsample = i_downsample
        self.stride = stride
        self.relu = nn.ReLU()

    def forward(self, x):
      identity = x.clone()

      x = self.relu(self.batch_norm2(self.conv1(x)))
      x = self.batch_norm2(self.conv2(x))

      if self.i_downsample is not None:
          identity = self.i_downsample(identity)
      print(x.shape)
      print(identity.shape)
      x += identity
      x = self.relu(x)
      return x


        
        
class ResNet(nn.Module):
    def __init__(self, ResBlock, layer_list, num_classes, num_channels=3):
        super(ResNet, self).__init__()
        self.in_channels = 64
        
        self.conv1 = nn.Conv2d(num_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.batch_norm1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.max_pool = nn.MaxPool2d(kernel_size = 3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(ResBlock, layer_list[0], planes=64)
        self.layer2 = self._make_layer(ResBlock, layer_list[1], planes=128, stride=2)
        self.layer3 = self._make_layer(ResBlock, layer_list[2], planes=256, stride=2)
        self.layer4 = self._make_layer(ResBlock, layer_list[3], planes=512, stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.dropout = nn.Dropout(p=0.5)
        self.fc = nn.Linear(512*ResBlock.expansion, num_classes)
        
    def forward(self, x):
        x = self.relu(self.batch_norm1(self.conv1(x)))
        x = self.max_pool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.dropout(x)
        x = self.fc(x)
        
        return x
        
    def _make_layer(self, ResBlock, blocks, planes, stride=1):
        ii_downsample = None
        layers = []
        
        if stride != 1 or self.in_channels != planes*ResBlock.expansion:
            ii_downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, planes*ResBlock.expansion, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes*ResBlock.expansion)
            )
            
        layers.append(ResBlock(self.in_channels, planes, i_downsample=ii_downsample, stride=stride))
        self.in_channels = planes*ResBlock.expansion
        
        for i in range(blocks-1):
            layers.append(ResBlock(self.in_channels, planes))
            
        return nn.Sequential(*layers)

        
        
def ResNet50(num_classes, channels=3):
    return ResNet(Bottleneck, [3,4,6,3], num_classes, channels)

In [21]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels= 96, kernel_size= 11, stride=4, padding=0 )
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2)
        self.conv2 = nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, stride= 1, padding= 2)
        self.conv3 = nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, stride= 1, padding= 1)
        self.conv4 = nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.fc1  = nn.Linear(in_features= 9216, out_features= 4096)
        self.dropout1 = nn.Dropout(p=0.5)
        self.fc2  = nn.Linear(in_features= 4096, out_features= 4096)
        self.dropout2 = nn.Dropout(p=0.5)
        self.fc3 = nn.Linear(in_features=4096 , out_features=5)


    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = self.maxpool(x)
        x = F.relu(self.conv2(x))
        x = self.maxpool(x)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = self.maxpool(x)
        x = x.reshape(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

In [19]:
import cv2
from pytorch_grad_cam.utils.image import show_cam_on_image, \
    preprocess_image
import numpy as np
img_path = "/content/dataset_dr_preprocessed/c1/04ac765f91a1.png"
rgb_img = cv2.imread(img_path, 1)[:, :, ::-1]
rgb_img = cv2.resize(rgb_img, (224, 224))
rgb_img = np.float32(rgb_img) / 255
input_tensor = preprocess_image(rgb_img, mean=[0.5, 0.5, 0.5],
                                    std=[0.5, 0.5, 0.5])

## Gradcam

In [5]:
from pytorch_grad_cam import GradCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from torchvision.models import resnet50
import torch.nn as nn
model = ResNet50(5,3)
model = nn.DataParallel(model)
model.load_state_dict(torch.load("./ResNet50_Final2.pt"))
target_layers = [model.layer4[-1]]
#input_tensor = inp
# Create an input tensor image for your model..
# Note: input_tensor can be a batch tensor with several images!

# Construct the CAM object once, and then re-use it on many images:
cam = GradCAM(model=model, target_layers=target_layers, use_cuda=False)

# You can also use it within a with statement, to make sure it is freed,
# In case you need to re-create it inside an outer loop:
# with GradCAM(model=model, target_layers=target_layers, use_cuda=args.use_cuda) as cam:
#   ...

# We have to specify the target we want to generate
# the Class Activation Maps for.
# If targets is None, the highest scoring category
# will be used for every image in the batch.
# Here we use ClassifierOutputTarget, but you can define your own custom targets
# That are, for example, combinations of categories, or specific outputs in a non standard model.
targets = [ClassifierOutputTarget(0)]

# You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
grayscale_cam = cam(input_tensor=input_tensor, targets=targets)

# In this example grayscale_cam has only one image in the batch:
grayscale_cam = grayscale_cam[0, :]
visualization = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)
cv2.imwrite(f'0_grad_cam.jpg', visualization)

RuntimeError: ignored

In [30]:
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms, models
import torch
data_dir = "./dataset_dr_preprocessed/"
def train_val_dataset(dataset, val_split=0.20):
    train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=val_split,random_state=42)
    trainset = Subset(dataset, train_idx)
    testset = Subset(dataset, val_idx)
    return trainset,testset
transform = transforms.Compose([
    #transforms.ToPILImage(),
    #transforms.RandomHorizontalFlip(p=0.4),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])
dataset = datasets.ImageFolder(data_dir, transform=transform)
trainset,testset = train_val_dataset(dataset)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)
print(len(testset))


733


## Ensembling


In [ ]:
import torch.nn as nn
net1 = ResNet50(5,3)
net1 = nn.DataParallel(net1)
#net1.load_state_dict(torch.load("./ResNet50_Final2.pt"))
net2 = AlexNet()
#net2.load_state_dict(torch.load("./AlexNet_Final.pt"))
net3 = MyGoogleNet()
net3.load_state_dict(torch.load("./GoogleNet.pt"))

models =[net1,net2,net3]
correct = 0
total = 0
for batch_idx, (inputs, targets) in enumerate(testloader):
  image_tensor = inputs
  probs = []
  predicted = []
  outputs1 = models[0](image_tensor)
  outputs2 = models[1](image_tensor)
  outputs3 = models[2](image_tensor)

  outputs1 = outputs1.cpu().detach().numpy()
  outputs2 = outputs2.cpu().detach().numpy()
  outputs3 = outputs3.cpu().detach().numpy()

  for x in range(len(targets)):
    probs = [0,0,0,0,0]
    for i in range(0,len(probs)):
      #probs[i]+=outputs1[x][i]
      #probs[i]+=outputs2[x][i]
      probs[i]+=outputs3[x][i]

    avg_probs = [item/len(models) for item in probs]
    max_value = max(avg_probs)
    predicted=torch.tensor(avg_probs.index(max_value))
    print(predicted,targets[x])
    if predicted == targets[x]:
      correct+=1
    total+=1
  
  print(total,correct,targets)
  
  #print(correct,total)
acc = 100.*correct/total
print("test acc is  "+ str(acc))